# Baltimor Salary Prediction

To-Do : Dataset which captures the salary from July 1st, 2013 through June 30th, 2014. It includes only those employees who are employed on June 30, 2014. Predict the salary of Employees working in Baltimore

* Libraries

In [3]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error
 

* Importing Dataset

In [5]:
df=pd.read_csv('https://data.baltimorecity.gov/datasets/baltimore::baltimore-city-employee-salaries.csv')

# Data Exploration


In [6]:
df.head()

,lastName,middleInitial,firstName,jobClass,agencyName,agencyID,annualSalary,grossPay,hireDate,fiscalYear,ObjectId
0,Aaron,D,Kareem,UTILITIES INST REPAIR II,Public Works - Water & Waste Water,A50,34115.0,44557.99,2018-08-27,FY2021,1
1,Aaron,R,Lynda,ELECTION JUDGES REGULAR,D01,D01,0.0,185.00,2020-12-08,FY2021,2
2,Aaron,G,Patricia,FACILITIES/OFFICE SERVICES II,Mayor's Office of Employment Development,A03,63457.0,29461.96,1979-10-24,FY2021,3
3,Abadir,O,Adam,OPERATIONS OFFICER II,Health Department,A65,74299.0,85450.35,2016-12-12,FY2021,4
4,Abaku,O,Aigbolosimuan,POLICE OFFICER EID,Police Department,A99,68226.0,102775.97,2018-04-17,FY2021,5


In [9]:
df.columns

Index(['lastName', 'middleInitial', 'firstName', 'jobClass', 'agencyName',
       'agencyID', 'annualSalary', 'grossPay', 'hireDate', 'fiscalYear',
       'ObjectId'],
      dtype='object')

In [10]:
df.dtypes

lastName          object
middleInitial     object
firstName         object
jobClass          object
agencyName        object
agencyID          object
annualSalary     float64
grossPay         float64
hireDate          object
fiscalYear        object
ObjectId           int64
dtype: object

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154187 entries, 0 to 154186
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   lastName       153777 non-null  object 
 1   middleInitial  121195 non-null  object 
 2   firstName      154187 non-null  object 
 3   jobClass       154003 non-null  object 
 4   agencyName     154187 non-null  object 
 5   agencyID       154187 non-null  object 
 6   annualSalary   154187 non-null  float64
 7   grossPay       154187 non-null  float64
 8   hireDate       154185 non-null  object 
 9   fiscalYear     154187 non-null  object 
 10  ObjectId       154187 non-null  int64  
dtypes: float64(2), int64(1), object(8)
memory usage: 12.9+ MB


In [14]:
df.count()

lastName         153777
middleInitial    121195
firstName        154187
jobClass         154003
agencyName       154187
agencyID         154187
annualSalary     154187
grossPay         154187
hireDate         154185
fiscalYear       154187
ObjectId         154187
dtype: int64

In [15]:
df.describe()

,annualSalary,grossPay,ObjectId
count,154187.000000,154187.000000,154187.000000
mean,52873.341579,55266.226727,77094.000000
std,25066.781786,33516.800966,44510.097315
min,0.000000,1.710000,1.000000
25%,34518.000000,33404.750000,38547.500000
50%,50238.000000,52070.440000,77094.000000
75%,68487.000000,74476.905000,115640.500000
max,276375.000000,373111.190000,154187.000000


In [16]:
df.shape

(154187, 11)

In [30]:
df.tail()

,lastName,middleInitial,firstName,jobClass,agencyName,agencyID,annualSalary,grossPay,hireDate,fiscalYear,ObjectId
154182,NaN,9,BPD,BPD,Police Department,A99,65499.0,64127.83,1970-01-01,FY2011,154183
154183,NaN,0,BPD,BPD,Police Department,A99,63384.0,66187.45,1970-01-01,FY2011,154184
154184,NaN,1,BPD,BPD,Police Department,A99,61102.0,71989.51,1970-01-01,FY2011,154185
154185,NaN,2,BPD,BPD,Police Department,A99,59958.0,53235.59,1970-01-01,FY2011,154186
154186,NaN,3,BPD,BPD,Police Department,A99,68523.0,95152.89,1970-01-01,FY2011,154187


In [31]:
newdf=df.copy()

In [33]:
newdf.columns=newdf.columns.str.strip().str.lower()

In [35]:
newdf.head()

,lastname,middleinitial,firstname,jobclass,agencyname,agencyid,annualsalary,grosspay,hiredate,fiscalyear,objectid
0,Aaron,D,Kareem,UTILITIES INST REPAIR II,Public Works - Water & Waste Water,A50,34115.0,44557.99,2018-08-27,FY2021,1
1,Aaron,R,Lynda,ELECTION JUDGES REGULAR,D01,D01,0.0,185.00,2020-12-08,FY2021,2
2,Aaron,G,Patricia,FACILITIES/OFFICE SERVICES II,Mayor's Office of Employment Development,A03,63457.0,29461.96,1979-10-24,FY2021,3
3,Abadir,O,Adam,OPERATIONS OFFICER II,Health Department,A65,74299.0,85450.35,2016-12-12,FY2021,4
4,Abaku,O,Aigbolosimuan,POLICE OFFICER EID,Police Department,A99,68226.0,102775.97,2018-04-17,FY2021,5


In [42]:
newdf.isnull().sum()

lastname           410
middleinitial    32992
firstname            0
jobclass           184
agencyname           0
agencyid             0
annualsalary         0
grosspay             0
hiredate             2
fiscalyear           0
objectid             0
dtype: int64

In [44]:
newdf.drop('grosspay',axis=1,inplace=True)

In [46]:
newdf.columns

Index(['lastname', 'middleinitial', 'firstname', 'jobclass', 'agencyname',
       'agencyid', 'annualsalary', 'hiredate', 'fiscalyear', 'objectid'],
      dtype='object')

* Duplicated of the Agency Id

In [52]:
newdf.agencyid.value_counts()

A99    34706
A64    18374
A50    16353
A65     9640
A49     7007
       ...  
A28        3
P65        3
A53        2
U01        1
A9         1
Name: agencyid, Length: 71, dtype: int64

In [56]:
newdf.agencyid.value_counts().map(newdf.agencyname)

A99                   State's Attorney's Office
A64                                         D01
A50                           Health Department
A65                                       MOCFS
A49                             Fire Department
                         ...                   
A28                           Health Department
P65                           Health Department
A53    Mayor's Office of Employment Development
U01                                         D01
A9                                          D01
Name: agencyid, Length: 71, dtype: object

In [62]:
newdf.jobclass.value_counts()

POLICE OFFICER                    18958
CROSSING GUARD                     3546
LABORER (HOURLY)                   3388
EMT FIREFIGHTER SUPPRESSION        2876
COMMUNITY AIDE                     2865
                                  ...  
QUALITY ASSURANCE ANALYST SUPE        1
FIRE LT. INVES & PRE SERV, ALS        1
CUSTOMER CARE ANALYST SUP II          1
SPECIAL ASSIST. TO THE STATE'S        1
COURT REPORTER                        1
Name: jobclass, Length: 1897, dtype: int64

In [63]:
newdf.annualsalary.value_counts()

0.0        1958
20800.0    1447
24960.0    1212
22880.0    1030
15080.0     847
           ... 
89126.0       1
84708.0       1
82895.0       1
58337.0       1
48006.0       1
Name: annualsalary, Length: 13321, dtype: int64

In [66]:
newdf.annualsalary.astype(int)

0         34115
1             0
2         63457
3         74299
4         68226
          ...  
154182    65499
154183    63384
154184    61102
154185    59958
154186    68523
Name: annualsalary, Length: 154187, dtype: int32

In [67]:
newdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154187 entries, 0 to 154186
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   lastname       153777 non-null  object 
 1   middleinitial  121195 non-null  object 
 2   firstname      154187 non-null  object 
 3   jobclass       154003 non-null  object 
 4   agencyname     154187 non-null  object 
 5   agencyid       154187 non-null  object 
 6   annualsalary   154187 non-null  float64
 7   hiredate       154185 non-null  object 
 8   fiscalyear     154187 non-null  object 
 9   objectid       154187 non-null  int64  
dtypes: float64(1), int64(1), object(8)
memory usage: 11.8+ MB


In [68]:
newdf.head()

,lastname,middleinitial,firstname,jobclass,agencyname,agencyid,annualsalary,hiredate,fiscalyear,objectid
0,Aaron,D,Kareem,UTILITIES INST REPAIR II,Public Works - Water & Waste Water,A50,34115.0,2018-08-27,FY2021,1
1,Aaron,R,Lynda,ELECTION JUDGES REGULAR,D01,D01,0.0,2020-12-08,FY2021,2
2,Aaron,G,Patricia,FACILITIES/OFFICE SERVICES II,Mayor's Office of Employment Development,A03,63457.0,1979-10-24,FY2021,3
3,Abadir,O,Adam,OPERATIONS OFFICER II,Health Department,A65,74299.0,2016-12-12,FY2021,4
4,Abaku,O,Aigbolosimuan,POLICE OFFICER EID,Police Department,A99,68226.0,2018-04-17,FY2021,5


In [69]:
newdf.hiredate[0]

'2018-08-27'

In [76]:
date_clean=re.sub("^\d{4}.*-","",newdf.hiredate[0])
date_clean

'27'

In [80]:
month_clean=re.sub("^\d{4}-|-\d{2}","",newdf.hiredate[0])
month_clean

'08'

In [82]:
year_clean=re.sub("-\d{2}.*","",newdf.hiredate[0])
year_clean

'2018'

In [84]:
newdf.shape[0]

154187

In [86]:
newdff=newdf.copy()

In [ ]:
# for i in range(len(newdff)):
for i in range(154187):
    newdff['hiredate'][i]=re.sub("^\d{4}.*-","",newdf.hiredate[i])
    newdff['hiremonth'][i]=re.sub("^\d{4}-|-\d{2}","",newdf.hiredate[i])
    newdff['hireyear'][i]=re.sub("-\d{2}.*","",newdf.hiredate[i])
    